### PROYECTO INTEGRADOR

In [4]:
import csv
import pandas as pd
import numpy as np

In [5]:
datosp=pd.read_csv('C:/Users/Acer/Downloads/bd_clean2.csv',sep=';', index_col=0) #Carga datos
datosp.head()

,Sexo,Estrato,Grupo_valor,Rango_ingresos_acum,Nivel_estudio,Estado_civil,Departamento,Edad,Franquicia,canal,...,cnt_trx_201811,cnt_trx_201812,cnt_trx_201901,cnt_trx_201902,cnt_trx_201903,cnt_trx_201904,cnt_trx_201905,cnt_trx_201906,Total_trx,Recency
Cliente_Id,,,,,,,,,,,,,,,,,,,,,
45653157,F,5,Alto,28500000,Especializacion,VIUDO,VALLE DEL CAUCA,79,MASTER,I,...,1,0,0,3,0,0,1,0,218,1
25037319,M,4,Alto,21000000,Especializacion,VIUDO,ANTIOQUIA,77,MASTER,P,...,0,12,0,0,0,0,0,0,213,6
48475523,F,4,Medio,3000000,Pregrado,VIUDO,BOGOTA D.C.,77,VISA,I,...,2,1,2,1,0,4,0,3,57,0
46339223,F,5,Alto,22500000,Pregrado,VIUDO,BOGOTA D.C.,76,AMEX,I,...,12,8,2,0,0,0,0,0,165,5
21539592,F,6,Alto,13500000,Especializacion,VIUDO,ANTIOQUIA,76,VISA,P,...,0,0,0,0,0,0,0,0,124,8


Para crear el modelo FMR, es necesario identificar los  percentiles a los que  pertenece cada cliente, viendo de manera individual su frecuencia,monto trasado,y Recencia(tiempo transcurrido en meses desde su última compra).

In [16]:
Trx=datosp.Total_trx.quantile([0.2,0.4,0.6,0.8,1]).values.tolist()
Monto=datosp.Monto_transado.quantile([0.2,0.4,0.6,0.8,1]).values.tolist()
Rec=datosp.Recency.quantile([0.2,0.4,0.6,0.8,1]).values.tolist()

Definimos  una calificación a cada cliente por  pertenecer a cada  uno de los  rangos  del percentil

In [17]:
Frecuency=[]
Money=[]
Rcency=[]

for i in range(0,len(datosp)):
    if np.array(datosp.Total_trx)[i]<=Trx[0]:
        F=1
    elif np.array(datosp.Total_trx)[i]<=Trx[1]:
        F=2
    elif np.array(datosp.Total_trx)[i]<=Trx[2]:  
        F=3
    elif np.array(datosp.Total_trx)[i]<=Trx[3]: 
        F=4
    else:
        F=5
    Frecuency.append(F)

for i in range(0,len(datosp)):
    if np.array(datosp.Monto_transado)[i]<=Monto[0]:
        M=1
    elif np.array(datosp.Monto_transado)[i]<=Monto[1]:
        M=2
    elif np.array(datosp.Monto_transado)[i]<=Monto[2]:  
        M=3
    elif np.array(datosp.Monto_transado)[i]<=Monto[3]:
        M=4
    else:
        M=5
    Money.append(M)
    
for i in range(0,len(datosp)):
    if np.array(datosp.Recency)[i]<=Rec[0]:
        R=5
    elif np.array(datosp.Recency)[i]<=Rec[1]:
        R=4
    elif np.array(datosp.Recency)[i]<=Rec[2]:  
        R=3
    elif np.array(datosp.Recency)[i]<=Rec[3]: 
        R=2
    else:
        R=1
    Rcency.append(R)

datosp['Frecuency']=Frecuency
datosp['Money']=Money
datosp['Rcency']=Rcency

Calculamos el score RFM

In [18]:
datosp['RFM']=datosp.Frecuency*datosp.Money*datosp.Rcency
datosp.head()

,Sexo,Estrato,Grupo_valor,Rango_ingresos_acum,Nivel_estudio,Estado_civil,Departamento,Edad,Franquicia,canal,...,cnt_trx_201904,cnt_trx_201905,cnt_trx_201906,Total_trx,Recency,Frecuency,Money,Rcency,RFM,Percentil_RFM
Cliente_Id,,,,,,,,,,,,,,,,,,,,,
45653157,F,5,Alto,28500000,Especializacion,VIUDO,VALLE DEL CAUCA,79,MASTER,I,...,0,1,0,218,1,5,3,3,45,4
25037319,M,4,Alto,21000000,Especializacion,VIUDO,ANTIOQUIA,77,MASTER,P,...,0,0,0,213,6,4,3,3,36,4
48475523,F,4,Medio,3000000,Pregrado,VIUDO,BOGOTA D.C.,77,VISA,I,...,4,0,3,57,0,2,4,5,40,4
46339223,F,5,Alto,22500000,Pregrado,VIUDO,BOGOTA D.C.,76,AMEX,I,...,0,0,0,165,5,4,2,3,24,3
21539592,F,6,Alto,13500000,Especializacion,VIUDO,ANTIOQUIA,76,VISA,P,...,0,0,0,124,8,4,3,2,24,3


Calculamos  los percentiles del score RFM

In [20]:
RFM_perc=datosp.RFM.quantile([0.2,0.4,0.6,0.8,1]).values.tolist()
RFM_perc

[6.0, 16.0, 30.0, 60.0, 125.0]

In [21]:
Percentil=[]
for i in range(0,len(datosp)):
    if np.array(datosp.RFM)[i]<=RFM_perc[0]:
        P=1
    elif np.array(datosp.RFM)[i]<=RFM_perc[1]:
        P=2
    elif np.array(datosp.RFM)[i]<=RFM_perc[2]:  
        P=3
    elif np.array(datosp.RFM)[i]<=RFM_perc[3]: 
        P=4
    else:
        P=5
    Percentil.append(P)
datosp['Percentil_RFM']=Percentil

,Sexo,Estrato,Grupo_valor,Rango_ingresos_acum,Nivel_estudio,Estado_civil,Departamento,Edad,Franquicia,canal,...,cnt_trx_201904,cnt_trx_201905,cnt_trx_201906,Total_trx,Recency,Frecuency,Money,Rcency,RFM,Percentil_RFM
Cliente_Id,,,,,,,,,,,,,,,,,,,,,
45653157,F,5,Alto,28500000,Especializacion,VIUDO,VALLE DEL CAUCA,79,MASTER,I,...,0,1,0,218,1,5,3,3,45,4
25037319,M,4,Alto,21000000,Especializacion,VIUDO,ANTIOQUIA,77,MASTER,P,...,0,0,0,213,6,4,3,3,36,4
48475523,F,4,Medio,3000000,Pregrado,VIUDO,BOGOTA D.C.,77,VISA,I,...,4,0,3,57,0,2,4,5,40,4
46339223,F,5,Alto,22500000,Pregrado,VIUDO,BOGOTA D.C.,76,AMEX,I,...,0,0,0,165,5,4,2,3,24,3
21539592,F,6,Alto,13500000,Especializacion,VIUDO,ANTIOQUIA,76,VISA,P,...,0,0,0,124,8,4,3,2,24,3
